# File Search Demo
Beispiel für die Verwendung der OpenAI Assistant API und des File Search-Features.

## Setup

In [1]:
!pip install -qU openai==1.52.2

## API konfigurieren

In [2]:
import openai
import os
import time

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)

## Datei hochladen

In [5]:
# Erstellen eines Vector Stores
vector_store = client.beta.vector_stores.create(name="NeoGeeksVectorStore")

# Pfade zu den Dateien, die hochgeladen werden sollen
dateipfade = ["/content/NeoGeeksAGB.pdf", "NeoGeeksAGB.pdf"]

# Hochladen der Dateien
dateien = [open(dateipfad, "rb") for dateipfad in dateipfade]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=dateien
)

## Assistant erstellen

In [6]:
assistant = client.beta.assistants.create(
    name="AGB Assistant",
    instructions="Du bist ein Assistant, der Rechtsfragen basierend auf dem bereitgestellten Dokument beantwortet.",
    model="gpt-4-turbo",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

## Thread erstellen

In [7]:
thread = client.beta.threads.create()

## Anweisung absetzen

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Wie hoch ist das Zahlungsziel von NeoGeeks?"
)

## Run erstellen, ausführen und Ergebnis ausgeben

In [9]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Abrufen der Antwortnachrichten
messages = client.beta.threads.messages.list(thread.id)
for msg in messages.data:
    if msg.role == "assistant":
        print("Assistant:", msg.content[0].text.value)

Assistant: Das Zahlungsziel von NeoGeeks beträgt 30 Kalendertage ab Rechnungserhalt【4:0†source】.
